Bulk of content is from [this](https://dbader.org/blog/python-dunder-methods) tutorial

We illustrate the importance of dunder (double under) methods using an Account class. First is defining a dunder method for **Object Initialisation**: `__init__`

In [1]:
class Account:
    """A simple account class"""

    def __init__(self, owner, amount=0):
        """
        This is the constructor that lets us create
        objects from this class
        """
        self.owner = owner
        self.amount = amount
        self._transactions = []

**Object Representation:** 
* `__str__`: For the end-user, the "informal" or nicely printable string representation of an object
* `__repr__`: The "official" to-string method of an object.

Below, we also access the class name `Account` programatically instead of hardcoding it into the to-string methods

In [8]:
class Account:
    """A simple account class"""

    def __init__(self, owner, amount=0):
        """
        This is the constructor that lets us create
        objects from this class
        """
        self.owner = owner
        self.amount = amount
        self._transactions = []
    
    def __repr__(self):
        return '{!r}({!r}, {!r})'.format(self.__class__.__name__,
                                         self.owner, self.amount)
    
    def __str__(self):
        return '{} of {} with starting amount: {}'.format(self.__class__.__name__,
                                         self.owner, self.amount)

In [9]:
acc = Account("bob", 10)

In [10]:
# __repr__ output:
acc

'Account'('bob', 10)

In [11]:
# __str__ output:
print(acc)

Account of bob with starting amount: 10


**Object Iteration:**
* `__len__`
* `__getitem__`
* `__reversed__`

Before we iterate on our account object, we first need to add transactions to the account. So we define a simple method `add_transaction` to add transactions, then a property `balance` to access the account balance before/after those transactions are processed

In [17]:
class Account:
    """A simple account class"""

    def __init__(self, owner, amount=0):
        """
        This is the constructor that lets us create
        objects from this class
        """
        self.owner = owner
        self.amount = amount
        self._transactions = []
    
    def __repr__(self):
        return '{!r}({!r}, {!r})'.format(self.__class__.__name__,
                                         self.owner, self.amount)
    
    def __str__(self):
        return '{} of {} with starting amount: {}'.format(self.__class__.__name__,
                                         self.owner, self.amount)
    
    def add_transaction(self, amount):
        """each transaction has to go into __transactions attribute"""
        if not isinstance(amount, int):
            raise ValueError('Please use int for amount')
        self._transactions.append(amount)
    
    @property
    def balance(self):
        return self.amount + sum(self._transactions)

In [37]:
# we try deposit and withdraw, then get balance from account
acc = Account('Bob', 10)

acc.add_transaction(20)
acc.add_transaction(-10)
acc.add_transaction(50)
acc.add_transaction(-10)
acc.add_transaction(30)

acc.balance

90

Now that we have transactions, what if i want to know:
1. How many transactions were there?
2. List all of the transactions
3. Get the nth transaction (i.e. 1st, 3rd, so forth)

Dunder methods to the rescue! (Lack of them --> `TypeError`). The appropriate dunder methods to make the 3 above possible are:
1. `__len__`
2. `__getitem__`
3. `__getitem__`

In [36]:
class Account:
    """A simple account class"""

    def __init__(self, owner, amount=0):
        """
        This is the constructor that lets us create
        objects from this class
        """
        self.owner = owner
        self.amount = amount
        self._transactions = []
    
    def __repr__(self):
        return '{!r}({!r}, {!r})'.format(self.__class__.__name__,
                                         self.owner, self.amount)
    
    def __str__(self):
        return '{} of {} with starting amount: {}'.format(self.__class__.__name__,
                                         self.owner, self.amount)
    
    # MORE Dunder methods
    def __len__(self):
        return len(self._transactions)
    
    def __getitem__(self, position):
        return self._transactions[position]
    
    def add_transaction(self, amount):
        """each transaction has to go into __transactions attribute"""
        if not isinstance(amount, int):
            raise ValueError('Please use int for amount')
        self._transactions.append(amount)
    
    @property
    def balance(self):
        return self.amount + sum(self._transactions)    

In [38]:
len(acc)

5

In [30]:
for transaction in acc:
    print(transaction)

20
-10
50
-10
30


In [40]:
acc[4]

30